# Caching

O LangChain fornece a possibilidade de salvar inputs/outputs em caching para que o modelo possa responder rapidamente e diminuir custos de requisição de API.

## Setup

In [1]:
import hashlib
import json

from dotenv import find_dotenv, load_dotenv
from langchain_openai import OpenAI
from langchain_core.caches import InMemoryCache
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache

_ = load_dotenv(find_dotenv('.env'))

## Memória

In [2]:
openai = OpenAI(model='gpt-3.5-turbo-instruct')
set_llm_cache(InMemoryCache())

In [3]:
PROMPT = 'Me diga em poucas palavras quem foi Carl Sagan'

In [4]:
response_without_cache = openai.invoke(PROMPT)

print('Resposta sem cache:', response_without_cache)

Resposta sem cache: 

Carl Sagan foi um renomado astrônomo, astrofísico, cosmólogo, escritor e divulgador científico americano, conhecido por popularizar a ciência e a exploração espacial através de seus livros, programas de televisão e palestras. Ele também foi um defensor da pesquisa científica e da busca por vida extraterrestre. Sua obra mais famosa é a série de TV "Cosmos", que se tornou um marco na divulgação científica. Sagan também foi um ativista ambiental e crítico da pseudociência e do uso indevido da ciência. Ele faleceu em 1996, deixando um legado duradouro na comunidade científica e na sociedade em geral.


In [5]:
response_with_cache = openai.invoke(PROMPT)
print('Resposta com cache:', response_with_cache)

Resposta com cache: 

Carl Sagan foi um renomado astrônomo, astrofísico, cosmólogo, escritor e divulgador científico americano, conhecido por popularizar a ciência e a exploração espacial através de seus livros, programas de televisão e palestras. Ele também foi um defensor da pesquisa científica e da busca por vida extraterrestre. Sua obra mais famosa é a série de TV "Cosmos", que se tornou um marco na divulgação científica. Sagan também foi um ativista ambiental e crítico da pseudociência e do uso indevido da ciência. Ele faleceu em 1996, deixando um legado duradouro na comunidade científica e na sociedade em geral.


## Disco/Banco de Dados

In [6]:
set_llm_cache(SQLiteCache(database_path='cache.db'))

In [7]:
PROMPT = 'Me diga em poucas palavras quem foi Neil Armstrong'

In [8]:
response_without_cache = openai.invoke(PROMPT)
print('Resposta sem cache:', response_without_cache)

Resposta sem cache: 

Neil Armstrong foi um astronauta e engenheiro aeroespacial americano que se tornou o primeiro ser humano a pisar na Lua em 1969 como parte da missão Apollo 11. Ele é considerado um herói nacional e um pioneiro na exploração espacial. 


In [9]:
response_with_cache = openai.invoke(PROMPT)
print('Resposta com cache:', response_with_cache)

Resposta com cache: 

Neil Armstrong foi um astronauta e engenheiro aeroespacial americano que se tornou o primeiro ser humano a pisar na Lua em 1969 como parte da missão Apollo 11. Ele é considerado um herói nacional e um pioneiro na exploração espacial. 


## Personalizado

In [28]:
import hashlib
import json
from pathlib import Path

from langchain_core.caches import BaseCache
from langchain_core.load.dump import dumps
from langchain_core.load.load import loads
from langchain_core.outputs import Generation

class SimpleDiskCache(BaseCache):
    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)

    def _get_cache_path(self, prompt):
        """Cria nome do arquivo de cache único."""
        hashed_key = hashlib.md5(prompt.encode()).hexdigest()

        return self.cache_dir / f'{hashed_key}.json'

    def lookup(self, prompt, llm_string):
        cache_path = self._get_cache_path(prompt)

        if cache_path.exists():
            cached_response = cache_path.read_text()

            return loads(cached_response)
        return None

    def update(self, prompt, llm_string, return_val):
        cache_path = self._get_cache_path(prompt)
        cache_path.write_text(dumps(return_val))

    def clear(self, prompt):
        cache_path = self._get_cache_path(prompt)
        cache_path.unlink()

In [29]:
set_llm_cache(SimpleDiskCache())
PROMPT = 'Me diga quem foi Neil Armstrong'

In [30]:
response_without_cache = openai.invoke(PROMPT)
print('Resposta sem cache:', response_without_cache)

Resposta sem cache: 

Neil Armstrong foi um astronauta e engenheiro aeroespacial norte-americano, conhecido por ser o primeiro ser humano a pisar na Lua, durante a missão Apollo 11 em 1969. Ele nasceu em 5 de agosto de 1930 em Ohio, nos Estados Unidos, e faleceu em 25 de agosto de 2012. Além de sua famosa conquista na Lua, Armstrong também foi piloto de testes da Força Aérea dos EUA e participou de outras missões espaciais, como a Gemini 8. Ele é considerado um dos maiores heróis da exploração espacial e um símbolo de coragem e determinação. 


In [31]:
response_with_cache = openai.invoke(PROMPT)
print('Resposta com cache:', response_with_cache)

Resposta com cache: 

Neil Armstrong foi um astronauta e engenheiro aeroespacial norte-americano, conhecido por ser o primeiro ser humano a pisar na Lua, durante a missão Apollo 11 em 1969. Ele nasceu em 5 de agosto de 1930 em Ohio, nos Estados Unidos, e faleceu em 25 de agosto de 2012. Além de sua famosa conquista na Lua, Armstrong também foi piloto de testes da Força Aérea dos EUA e participou de outras missões espaciais, como a Gemini 8. Ele é considerado um dos maiores heróis da exploração espacial e um símbolo de coragem e determinação. 
